#Projet Mise en oeuvre d'un Réseau Antagoniste Génératif (GAN) pour la création de visages


---


Marius Ferrieres, Bastien Garnier, Corentin Boudaud

Projet de 4ème Année SAGI (Système Automatisés et Génie Informatique)

Polytech Angers, 2021

---



In [ ]:
!pip install google-api-python-client
!pip install --version 3.1 oauth2client

In [ ]:
# Pour faire des gifs
!pip install -q imageio


!pip install -q git+https://github.com/tensorflow/docs
!pip install PyDrive

In [ ]:
#Librairies nécessaires
import tensorflow as tf
from tensorflow.keras import layers

import imageio

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import os
import cv2
import PIL
import time
import random
import oauth2client.contrib
from IPython import display

from zipfile import ZipFile
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('drive')

Mounted at drive


In [ ]:
# BLOCK ZIP-FILE

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"0B7EVK8r0v71pZjFTYXZWM3FlRnM"})
downloaded.GetContentFile('img_align_celeba.zip')

# Create a ZipFile Object and load sample.zip in it
with ZipFile('img_align_celeba.zip', 'r') as zipObj:
  # Get a list of all archived file names from the zip
  listOfFileNames = zipObj.namelist()
  for fileName in listOfFileNames:
    zipObj.extract(fileName)
print("Done")

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Done


In [ ]:
def getImages(number):
  # BLOCK CREATION DE LA BDD
  bddImg = []
  for a in range(number):
    randval = random.randint(1, len(listOfFileNames)) #200000
    for b in listOfFileNames[randval:randval + 1]:
      bddImg.append(mpl.image.imread(b))

  train_img = []
  for i in range(number):
    train_img.append(bddImg[i][0:216,0:176,:])
  train_img = np.array(train_img).astype('float32')
  train_img = (train_img[:,:,:,:] - 127)/127

  return train_img

In [ ]:
# BLOCK CREATION DE LA BDD
"""
bddImg = []
for a in range(0, 1000):
  randval = random.randint(1, len(listOfFileNames)) #200000
  for b in listOfFileNames[randval:randval + 1]:
    bddImg.append(mpl.image.imread(b))
print("Done")
"""

Done


In [ ]:
# ALL FUNCTIONS

def our_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(27*22*3, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((27, 22, 3)))
    print(model.output_shape)
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    print(model.output_shape)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    print(model.output_shape)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    print(model.output_shape)
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    print(model.output_shape)
    #assert model.output_shape == (None, 216, 176, 3)
    return model

def our_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[216, 176, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)
      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return


def train(epochs, number):
  for epoch in range(epochs):
    start = time.time()
    dataset = getImages(number)
    train_step(dataset)
    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    if (epoch + 1) % 20 == 0:
      generate_and_save_images(generator, epoch + 1, seed)
    # Save the model every 100 epochs
    if SAVE == 1:
      if (epoch + 1) % 100 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix, options=local_device_option)
  # Generate after the final epoch
  display.clear_output(wait=True)
  return

"""
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch in dataset:
      send = []
      send.append(image_batch)
      train_step(send)
    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    if (epoch + 1) % 5 == 0:
      generate_and_save_images(generator, epoch + 1, seed)
    # Save the model every 15 epochs
    if (epoch + 1) % 100 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
  # Generate after the final epoch
  display.clear_output(wait=True)
  return
"""

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(np.squeeze(np.asarray(predictions[i])).astype('float32'))
      #plt.imshow(predictions[i, :, :,2])
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

  plt.imshow(np.squeeze(np.asarray(predictions[0])).astype('float32'))
  plt.axis('off')
  plt.show()
  return

def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

print("Done!")

Done!


In [ ]:
# BLOCK GENERATOR & DISCRIMINATOR
generator = our_generator_model()
noise = tf.random.normal([1, 100])

discriminator = our_discriminator_model()
print("Done!")

(None, 27, 22, 3)
(None, 27, 22, 3)
(None, 54, 44, 3)
(None, 108, 88, 3)
(None, 216, 176, 3)
Done!


In [ ]:
# BLOCK FORMATAGE IMAGES
"""
train_img = []
for i in range(0,len(bddImg)):
  train_img.append(bddImg[i][0:216,0:176,:])
train_img = np.array(train_img).astype('float32')
train_img = (train_img[:,:,:,:] - 127)/127
print("Done!")
"""

In [ ]:
# BLOCK VARIABLES IMPORTANTES
"""
BUFFER_SIZE = train_img.shape[0]
"""

SAVE = 1

BATCH_SIZE = 600
"""
train_dataset = tf.data.Dataset.from_tensor_slices(train_img).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
"""

EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])
print("Done!")

Done!


In [ ]:
# BLOCK OPTIMISATION
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
print("Done!")

Done!


In [ ]:
#Sauvegarde dans drive des checkpoints

# BLOCK CHECKPOINT
checkpoint_dir = '/content/drive/MyDrive/Gan_Save/Checkpoints_Gan'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
print("Done!")

Done!


In [ ]:
#RESTORE THE CHECKPOINT
#You have to upload the newest checkpoint in your directory
#Then:   -Delete everything in your directory EXCEPT the file named "checkpoints"
#        -Upload the new files
#        -In this file you have to replace the number after "ckpt" to 1
#        -Replace the number of both of the checkpoint files to 1
#You can now run this part

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir), options=local_device_option).expect_partial()

In [ ]:
# BLOCK TRAINING
#entraine le réseau
"""
train(train_dataset, EPOCHS)
"""
train(EPOCHS, BATCH_SIZE)
#affiche le résultat final
display_image(EPOCHS - 1)

Time for epoch 28 is 3.4003312587738037 sec


IndexError: ignored

#Création GIF

In [ ]:
import glob

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

###Intégration dans un site web

##Conclusion


##Remerciements


##Sources




### Vidéos Youtube :

Arxiv Insights, Face editing with Generative Adversarial Networks: https://www.youtube.com/watch?v=dCKbRCUyop8&ab_channel=ArxivInsights

Two Minutes Paper, StyleGAN2: Near-Perfect Human Face Synthesis...and More: https://www.youtube.com/watch?v=SWoravHhsUU&ab_channel=TwoMinutePapers

Henry AI Labs, StyleGANv2 Explained! : https://www.youtube.com/watch?v=u8qPvzk0AfY&ab_channel=HenryAILabs

###Banques d'image :

Large-scale CelebFaces Attributes (CelebA) Dataset : http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html (Plus de 10,000 personnes uniques et 200,000 photos)

###Papiers scientifiques :

*   

###Sites Web:

*   Tutoriel Réseau Adversaire Génératif Convolutionnel Profond : https://www.tensorflow.org/tutorials/generative/dcgan
*   What is a Generative Adversarial Network? : http://hunterheidenreich.com/blog/what-is-a-gan/